In [ ]:
from sklearn.datasets import load_breast_cancer
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import time
def SFS(features, labels, Names):
  Ns = features.shape[1]
  feature_selected = []
  accs = []
  best_acc = 0
  best_loc = 0
  LDA = LinearDiscriminantAnalysis()
  t_0 = time.time()
  for i in range(Ns):
    remaining = [feature for feature in range(Ns) if feature not in feature_selected]
    best_features_now  = None
    best_acc_now = 0
    feature_Combination = []
    for feat in remaining:
      curr_features = feature_selected + [feat]
      curr_features = [fea for fea in curr_features if fea is not None]
      x1,x2,y1,y2 = train_test_split(features[:, np.array(curr_features).astype(int)], labels, test_size = 0.5,random_state = 0)
      LDA.fit(x1,y1)
      acc1 = LDA.score(x2,y2)*100
      LDA.fit(x2,y2)
      acc2 = LDA.score(x1,y1)*100
      avg = 0.5*(acc1+acc2)
      if avg > best_acc_now:
        best_acc_now = avg
        best_features_now = feat
    if best_features_now is not None:
      feature_selected.append(best_features_now)
      accs.append(best_acc_now)
    if best_acc_now > best_acc:
      best_acc = best_acc_now
      best_loc = i
    print(f'Step no. {i+1}: Feature selected now {best_features_now}. Feature name: {Name[best_features_now]}')
    print('Associated accuracy = {:.2f}%'.format(best_acc_now))
  t_f = time.time()
  print("Best accuracy is: {:.2f}%" .format(best_acc))
  print("Features that have best accuracy (indices)", feature_selected[:best_loc])
  print("Time cost is {:.4f} sec(s).".format(t_f-t_0))
#Main starts
cancer = load_breast_cancer()
features = cancer.data
labels = cancer.target
Name = cancer.feature_names
SFS(features, labels, Name)

Step no. 1: Feature selected now 27. Feature name: worst concave points
Associated accuracy = 91.04%
Step no. 2: Feature selected now 20. Feature name: worst radius
Associated accuracy = 94.02%
Step no. 3: Feature selected now 1. Feature name: mean texture
Associated accuracy = 95.61%
Step no. 4: Feature selected now 21. Feature name: worst texture
Associated accuracy = 95.61%
Step no. 5: Feature selected now 23. Feature name: worst area
Associated accuracy = 96.49%
Step no. 6: Feature selected now 14. Feature name: smoothness error
Associated accuracy = 96.84%
Step no. 7: Feature selected now 28. Feature name: worst symmetry
Associated accuracy = 97.01%
Step no. 8: Feature selected now 15. Feature name: compactness error
Associated accuracy = 97.36%
Step no. 9: Feature selected now 3. Feature name: mean area
Associated accuracy = 97.54%
Step no. 10: Feature selected now 17. Feature name: concave points error
Associated accuracy = 97.54%
Step no. 11: Feature selected now 5. Feature nam

In [ ]:
from sklearn.datasets import load_breast_cancer
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
def Fisher_with_2fold(features,labels, Names, d=30):
  class1_len = len(features[labels == 1])
  class0_len = len(features[labels == 0])
  t_0 = time.time()
  mean_vec = []
  for i in np.unique(labels):
    mean_vec.append(np.mean(features[labels == i], axis = 0))
  mean = np.mean(mean_vec, axis = 0)
  Sw = np.zeros((d, d))
  for ix in np.unique(labels):
    x = features[labels == ix]
    for j in range(len(x)):
      temp = x[j, :] - mean_vec[ix]
      Sw += np.outer(temp,temp)
  Sb = np.zeros((d, d))
  for i in np.unique(labels):
      Sb += len(features[labels==i])*(mean_vec[i] - mean)@ (mean_vec[i] - mean).T
      Fscore = np.diag(np.linalg.inv(Sw) @ Sb)
      aftersort = np.argsort(Fscore)[::-1]
      top_d_feature = aftersort[:d]
      top_dscore = Fscore[top_d_feature]
  best_acc = 0
  best_features = None
  LDA = LinearDiscriminantAnalysis()
  for ix in range(1, min(d, top_d_feature.shape[0]) + 1):
    feature_selected = features[:, top_d_feature[:ix]]
    x1,x2,y1,y2 = train_test_split(feature_selected, labels, test_size = 0.5, random_state = 0)
    LDA.fit(x1,y1)
    acc1 = LDA.score(x2,y2)*100
    LDA.fit(x2,y2)
    acc2 = LDA.score(x1,y1)*100
    acc = (acc1+acc2)/2
    print(f'Top {ix} features, Selected feature now is {top_d_feature[ix-1]}')
    print(f'Feature name:{Names[top_d_feature[ix-1]]}')
    print("associated accuracy is: {:.2f}%".format(acc))
    if acc> best_acc:
      best_acc = acc
      best_features = feature_selected
  t_f = time.time()
  print("Best accuracy is: {:.2f}%".format(best_acc))
  print("Selected Features that have best accuracy:", top_d_feature[: best_features.shape[1]])
  print("Time cost is: {:.4f} sec(s).".format(t_f-t_0))
  return top_d_feature, top_dscore
cancer = load_breast_cancer()
features = cancer.data
labels = cancer.target
Fisher_with_2fold(features, labels, Name)

Top 1 features, Selected feature now is 19
Feature name:fractal dimension error
associated accuracy is: 61.68%
Top 2 features, Selected feature now is 14
Feature name:smoothness error
associated accuracy is: 61.51%
Top 3 features, Selected feature now is 17
Feature name:concave points error
associated accuracy is: 73.11%
Top 4 features, Selected feature now is 9
Feature name:mean fractal dimension
associated accuracy is: 72.23%
Top 5 features, Selected feature now is 4
Feature name:mean smoothness
associated accuracy is: 81.19%
Top 6 features, Selected feature now is 7
Feature name:mean concave points
associated accuracy is: 90.16%
Top 7 features, Selected feature now is 18
Feature name:symmetry error
associated accuracy is: 89.63%
Top 8 features, Selected feature now is 25
Feature name:worst compactness
associated accuracy is: 92.62%
Top 9 features, Selected feature now is 26
Feature name:worst concavity
associated accuracy is: 92.27%
Top 10 features, Selected feature now is 28
Featur

(array([19, 14, 17,  9,  4,  7, 18, 25, 26, 28, 20, 12, 21,  8, 13,  3, 23,
        22,  2,  1,  0, 11,  6, 10,  5, 27, 24, 16, 15, 29]),
 array([ 2.57041921e+11,  7.64530593e+10,  4.47200008e+10,  3.71724601e+10,
         5.46423160e+09,  4.11685775e+09,  4.09204119e+09,  3.93498890e+09,
         2.35713882e+09,  5.35530335e+08,  2.87341208e+08,  6.20930204e+07,
         4.21002620e+07,  1.84279654e+07,  9.43408346e+06,  9.03891645e+05,
        -1.47378883e+06, -3.61732437e+06, -1.93278954e+07, -3.83844556e+07,
        -5.47301524e+07, -2.31242194e+08, -2.08195417e+09, -2.14433227e+09,
        -4.42511433e+09, -4.86850836e+09, -9.11017993e+09, -1.17783088e+10,
        -3.25614773e+10, -3.61256980e+10]))

In [ ]:
from sklearn.datasets import load_breast_cancer
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

import time

# SFS
def sequential_forward_selection_2fold(X, y):

    print("------------------------SFS-----------------------------")

    num_features = X.shape[1]
    selected_features = []
    accuracies = []
    best_accuracy = 0
    best_loc = 0

    start_time = time.time()

    for i in range(num_features):   # 30
        remaining_features = [feature for feature in range(num_features) if feature not in selected_features]
        local_best_feature = None
        local_best_accuracy = 0

        for feature in remaining_features:
            current_features = selected_features + [feature]

            # Split the data into two folds for each iteration
            current_features = [feature for feature in current_features if feature is not None]
            fold1_X, fold2_X, fold1_y, fold2_y = train_test_split(X[:, np.array(current_features).astype(int)], y, test_size=0.5, random_state=0)

            # Initialize LDA
            lda_ = LDA()

            # Fit and evaluate the model on the first fold
            lda_.fit(fold1_X, fold1_y)
            fold_1_accuracy = lda_.score(fold2_X, fold2_y) * 100

            # Fit and evaluate the model on the second fold
            lda_.fit(fold2_X, fold2_y)
            fold_2_accuracy = lda_.score(fold1_X, fold1_y) * 100

            # Calculate the average accuracy
            accuracy = (fold_1_accuracy + fold_2_accuracy) / 2

            if accuracy > local_best_accuracy:
                local_best_accuracy = accuracy
                local_best_feature = feature

        if local_best_feature is not None:
            selected_features.append(local_best_feature)
            accuracies.append(local_best_accuracy)

        if local_best_accuracy > best_accuracy:
            best_accuracy = local_best_accuracy
            best_loc = i

        print(f" Step {i + 1}: Selected Feature {local_best_feature}, Accuracy: {local_best_accuracy}")

    end_time = time.time()
    execution_time = end_time - start_time
    print(f"time cost = {execution_time} seconds")

    print(" [-] Best Accuracy:", best_accuracy)
    print(" [-] Features with the highest accuracy (indices):", selected_features[:best_loc])

    return selected_features, accuracies


def fisher_criterion_2fold(X, y, k=30):

    print("------------------------Fisher's Criterion-----------------------------")

    pos_len = len(X[y == 1])
    neg_len = len(X[y == 0])
    data_len = (pos_len + neg_len)

    # find the mean vector for each class
    mean_vectors = []
    for i in np.unique(y):
        mean_vectors.append(np.mean(X[y == i], axis=0))

    mean = np.mean(mean_vectors, axis=0)

    # find the within-class matrix (Sw)
    Sw = np.zeros((X.shape[1], X.shape[1]))
    for i in np.unique(y):
        class_samples = X[y == i]
        for j in range(len(class_samples)):
            diff = class_samples[j, :] - mean_vectors[i]
            Sw += np.outer(diff, diff)

    # find the Between-class matrix (Sb)
    Sb = np.zeros((X.shape[1], X.shape[1]))
    for i in np.unique(y):
        Sb += len(X[y == i]) * (mean_vectors[i] - mean) @ (mean_vectors[i] - mean).T

        # calculate Fisher's scores
        F_scores = np.diag(np.linalg.inv(Sw) @ Sb)

        # sort indices based on F-scores in descending order
        sorted_indices = np.argsort(F_scores)[::-1]

        # get top k indices and their corresponding F-scores
        top_k_indices = sorted_indices[:k]
        top_k_scores = F_scores[top_k_indices]

        # print top k indices and their corresponding F-scores
        # print(f"Top {k} indices: {top_k_indices}")

    # Initialize variables to store information about the best accuracy
    best_accuracy = 0
    best_selected_features = None

    start_time = time.time()

    # Loop through the top k features
    for i in range(1, min(k, len(top_k_indices)) + 1):
        selected_features = X[:, top_k_indices[:i]]

        # Split the data into two folds for each iteration
        fold1_X, fold2_X, fold1_y, fold2_y = train_test_split(selected_features, y, test_size=0.5, random_state=0)

        # Initialize LDA
        lda_ = LDA()

        # Fit and evaluate the model on the first fold
        lda_.fit(fold1_X, fold1_y)
        fold_1_accuracy = lda_.score(fold2_X, fold2_y) * 100

        # Fit and evaluate the model on the second fold
        lda_.fit(fold2_X, fold2_y)
        fold_2_accuracy = lda_.score(fold1_X, fold1_y) * 100

        accuracy = (fold_1_accuracy + fold_2_accuracy) / 2

        print(f" Top {i} features, Selected Feature = {top_k_indices[i-1]},  Accuracy: {accuracy}")

        # Update the best accuracy and selected features if a new best is found
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_selected_features = selected_features

    end_time = time.time()
    execution_time = end_time - start_time
    print(f"time cost = {execution_time} seconds")

    # Print the best accuracy and corresponding features
    print(" [-] Best Accuracy:", best_accuracy)
    print(" [-] Selected Features for Best Accuracy:", top_k_indices[:best_selected_features.shape[1]])


    return top_k_indices, top_k_scores



def main():
    cancer = load_breast_cancer()
    x = cancer.data
    y = cancer.target

    sequential_forward_selection_2fold(x, y)
    fisher_criterion_2fold(x, y)


if __name__ == '__main__':
    main()


------------------------SFS-----------------------------
 Step 1: Selected Feature 27, Accuracy: 91.03904126513467
 Step 2: Selected Feature 20, Accuracy: 94.02458611317024
 Step 3: Selected Feature 1, Accuracy: 95.60723993081295
 Step 4: Selected Feature 21, Accuracy: 95.60723993081295
 Step 5: Selected Feature 23, Accuracy: 96.48566839634297
 Step 6: Selected Feature 14, Accuracy: 96.83654558932542
 Step 7: Selected Feature 28, Accuracy: 97.01198418581666
 Step 8: Selected Feature 15, Accuracy: 97.364096861873
 Step 9: Selected Feature 3, Accuracy: 97.53953545836421
 Step 10: Selected Feature 17, Accuracy: 97.53953545836421
 Step 11: Selected Feature 5, Accuracy: 97.53953545836421
 Step 12: Selected Feature 26, Accuracy: 97.53891771682729
 Step 13: Selected Feature 11, Accuracy: 97.53891771682729
 Step 14: Selected Feature 13, Accuracy: 97.36347912033605
 Step 15: Selected Feature 22, Accuracy: 97.18804052384482
 Step 16: Selected Feature 4, Accuracy: 97.01198418581666
 Step 17: Sele